In [1]:
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Load features
train_manual = np.load("featuresets/train_manual_features.npy")
test_manual = np.load("featuresets/test_manual_features.npy")

train_mobilenet = np.load("featuresets/train_mobilenet_features.npy")
test_mobilenet = np.load("featuresets/test_mobilenet_features.npy")

train_densenet = np.load("featuresets/train_densenet_features.npy")
test_densenet = np.load("featuresets/test_densenet_features.npy")

train_labels = np.load("featuresets/train_manual_labels.npy")  # All labels are aligned
test_labels = np.load("featuresets/test_manual_labels.npy")

# Check dimensions
print("Manual:", train_manual.shape)
print("MobileNet:", train_mobilenet.shape)
print("DenseNet:", train_densenet.shape)
print("Labels:", train_labels.shape)

# Combine features
X_train = np.hstack((train_manual, train_mobilenet, train_densenet))
X_test = np.hstack((test_manual, test_mobilenet, test_densenet))

print("Combined train shape:", X_train.shape)
print("Combined test shape:", X_test.shape)


Manual: (8400, 14)
MobileNet: (8400, 1280)
DenseNet: (8400, 1024)
Labels: (8400,)
Combined train shape: (8400, 2318)
Combined test shape: (1400, 2318)


In [3]:
from sklearn.ensemble import VotingClassifier

ensemble = VotingClassifier(
    estimators=[
        ('svm', SVC(probability=True, C=1.0, gamma='scale', random_state=42)),
        ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
        ('rf', RandomForestClassifier(n_estimators=150, random_state=42))
    ],
    voting='soft',  # soft = use predicted probabilities
    n_jobs=-1
)

ensemble.fit(X_train, train_labels)
y_pred = ensemble.predict(X_test)

from sklearn.metrics import classification_report, accuracy_score
print("\n [Ensemble Voting Results]")
print(classification_report(test_labels, y_pred))
print("Voting Ensemble Accuracy:", accuracy_score(test_labels, y_pred))



 [Ensemble Voting Results]
              precision    recall  f1-score   support

           0       0.74      0.69      0.71       200
           1       0.88      0.93      0.90       200
           2       0.67      0.65      0.66       200
           3       0.84      0.80      0.82       200
           4       0.89      0.88      0.88       200
           5       0.72      0.75      0.74       200
           6       0.66      0.70      0.68       200

    accuracy                           0.77      1400
   macro avg       0.77      0.77      0.77      1400
weighted avg       0.77      0.77      0.77      1400

Voting Ensemble Accuracy: 0.77
